In [9]:
import dill

from loader import *

import math
import numpy as np
import random
import time
import torch
from torch.autograd import Variable as Var
import torch.nn as nn
import torch.optim as optim

### Data Loader

In [2]:
class LengthAdaptiveIterator(object):
    """Max-batch-length padded data iterator."""
    
    def __init__(self, source, target):
        """Initializer.
        
        Args:
            source: a list of lists of indices (sublist: a sentence).
            target: same as above.
        """
        self.source = source
        self.target = target
        self.size = len(source)
        self.indices = list(range(self.size))
        self.pad_index = 0
        self.unk_index = 1
        self.start_symbol_index = 2
        self.end_symbol_index = 3
        self.cursor = 0
        self.epoch = 0
        self.shuffle()
        
    def shuffle(self):
        """Shuffle indices."""
        random.shuffle(self.indices)
        
    def add_start_and_end_symbols(self, sentence):
        """Append start and end symbols."""
        return [self.start_symbol_index] + sentence + [self.end_symbol_index]
        
    def get_batch(self, batch_indices):
        """Get a batch of raw data by given batch of indices."""
        batch_source, batch_target = [], []
        batch_source_length, batch_target_length = [], []
        for index in batch_indices:
            source = self.add_start_and_end_symbols(self.source[index])
            target = self.add_start_and_end_symbols(self.target[index])
            source_length, target_length = len(source), len(target)
            batch_source.append(source)
            batch_target.append(target)
            batch_source_length.append(source_length)
            batch_target_length.append(target_length)
        return batch_source, batch_target, \
               batch_source_length, batch_target_length
    
    def pad_sentence(self, sentence, length, pad_length):
        """Pad a sentence (of length `length`) to a specified `pad_length`."""
        sentence_padded = sentence[:pad_length-1] + [self.end_symbol_index] \
            if length > pad_length \
            else sentence + [self.pad_index] * (pad_length - length)
        return sentence_padded
    
    def pad_batch(self, batch_sentence, batch_length):
        """Pad a batch of sentences."""
        pad_length = max(batch_length)
        return [self.pad_sentence(sentence, length, pad_length)
                for sentence, length in zip(batch_sentence, batch_length)]
    
    def next_batch(self, batch_size, batch_first=False):
        """Get the next batch of data.
        
        If `batch_first`, data shape = <batch-size, max-source/target-length>.
        Else, <max-source/target-length, batch-size>.
        """
        if self.cursor + batch_size > self.size:
            self.cursor = 0
            self.epoch += 1
        batch_indices = self.indices[self.cursor:self.cursor+batch_size]
        self.cursor += batch_size
        batch_source, batch_target, \
        batch_source_length, batch_target_length = self.get_batch(batch_indices)
        batch_source_padded = self.pad_batch(batch_source, batch_source_length)
        batch_target_padded = self.pad_batch(batch_target, batch_target_length)
        sort_indices = [index for length, index in sorted(zip(batch_source_length,
                                                              range(batch_size)),
                                                          reverse=True)]
        batch_source_padded = Var(torch.LongTensor(np.array(batch_source_padded)[sort_indices]))
        batch_target_padded = Var(torch.LongTensor(np.array(batch_target_padded)[sort_indices]))
        batch_source_length = Var(torch.LongTensor(np.array(batch_source_length)[sort_indices]))
        batch_target_length = Var(torch.LongTensor(np.array(batch_target_length)[sort_indices]))
        if not batch_first:
            batch_source_padded = batch_source_padded.transpose(0, 1)
            batch_target_padded = batch_target_padded.transpose(0, 1)            
        return batch_source_padded, batch_target_padded, \
               batch_source_length, batch_target_length


##### TESTING AREA

In [203]:
data_dir = "Data/"

indexer, train_source, train_target, valid_source, valid_target = load_corpus(data_dir)
len(train_source), len(train_target), len(valid_source), len(valid_target)

... loading `Data/train_source.txt`
... loading `Data/train_target.txt`
... loading `Data/valid_source.txt`
... loading `Data/valid_target.txt`


(331163, 331163, 162023, 162023)

In [294]:
data_iter = LengthAdaptiveIterator(train_source, train_target)

In [295]:
s,t,sl,tl = data_iter.next_batch(3)

In [296]:
s.shape, t.shape, sl, tl

(torch.Size([13, 3]),
 torch.Size([14, 3]),
 tensor([ 13,  12,  12]),
 tensor([ 11,  11,  14]))

### Model

In [47]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class Embeddings(nn.Module):
    """Embedding lookup."""
    
    def __init__(self, embed_size, vocab_size, glove_init=None):
        """
        Args:
            embed_size: embedding size.
            vocab_size: vocab_size size.
            glove_init: numpy.ndarray, initializing embeddings.
        """
        super(Embeddings, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        if glove_init is not None:
            assert glove_init.shape == (vocab_size, embed_size)
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_init))
        self.embed_size = embed_size

    def forward(self, inputs):
        """
        Args:
            inputs: <batch-size, seq-length>.
        """
        # Lookup embeddings: <batch-size, seq-length> 
        #                 -> <batch-size, seq-length, embed-size>
        return self.embeddings(inputs) * math.sqrt(self.embed_size)


class GRUEncoder(nn.Module):
    """GRU encoder."""
    
    def __init__(self, input_size, output_size, 
                 number_layers=2, bidirectional=True, dropout_rate=0.1):
        """Initializer."""
        super(GRUEncoder, self).__init__()
        self.gru = nn.GRU(input_size=input_size,
                          hidden_size=output_size,
                          num_layers=number_layers,
                          dropout=dropout_rate if number_layers>1 else 0.,
                          bidirectional=bidirectional)
        self.number_layers = number_layers
        self.bidirectional = bidirectional
    
    def forward(self, batch_input, batch_length, return_memory=False):
        """Forwarding.
        
        Args:
            batch_input: torch.LongTensor, <max-encoder-length, batch-size, input-size>.
            batch_length: torch.LongTensor, <batch-size,>
        Returns:
            batch_h: torch.FloatTensor, <number-layers, batch-size, output-size> 
                     (output-size*2 if bidirectional).
        """
        # Pack inputs:
        #   IN: <seq-length, batch-size, embed-size>, <batch-size,>
        #   OUT: <full-non-zero-element-size, embed-size>, <max-non-zero-element-size>.
        batch_input_packed = nn.utils.rnn.pack_padded_sequence(batch_input, batch_length)
        # Run LSTM
        #   IN: packed tuple.
        #   OUT: packed tuple, (hidden-state, model-memory).
        #        hidden-state/model-memory: <number_layers*number_directions, batch-size, hidden-size>.
        batch_output_packed, batch_h = self.gru(batch_input_packed)
        if self.bidirectional:
            # Concate bidirectional outputs
            #   OUT: <number-layers, batch-size, hidden-size*2>.
            batch_h = torch.cat((batch_h[:self.number_layers, :, :], 
                                 batch_h[self.number_layers:, :, :]), dim=-1)
        if return_memory:
            # Get encoder memory: <max-non-zero-element-size, batch-size, hidden-size*number-directions>.
            batch_output, _ = nn.utils.rnn.pad_packed_sequence(batch_output_packed)
            return batch_output, batch_h
        return batch_h
    

class GRUDecoder(nn.Module):
    """GRU decoder"""
    
    def __init__(self, input_size, output_size,
                 number_layers=2, dropout_rate=0.1):
        """Initializer."""
        super(GRUDecoder, self).__init__()
        self.gru = nn.GRU(input_size=input_size,
                          hidden_size=output_size,
                          num_layers=number_layers,
                          dropout=dropout_rate if number_layers>1 else 0.,
                          bidirectional=False)
    
    def forward(self, batch_single_input, previous_h):
        """Forwarding.
        
        Args:
            batch_single_input: torch.LongTensor, <seq-length=1, batch-size, input-size>.
            previous_h: torch.FloatTensor, <number-layers, batch-size, output-size>.
        Returns:
            batch_single_output: torch.FloatTensor, <seq-length=1, batch-size, output-size>.
            this_h: torch.FloatTensor, same shape as `previous_h`.
        """
        # Decode one time step:
        #   IN: batch_single_input, <seq-length=1, batch-size, input-size>.
        #   OUT: batch_single_output, <seq-length=1, batch-size, output-size>.
        # previous_h keeps <number-layers, batch-size, output-size>.
        batch_single_output, this_h = self.gru(batch_single_input, previous_h)
        return batch_single_output, this_h
    
    
class VAE(nn.Module):
    """GRU seq2seq VAE."""
    
    def __init__(self, indexer, embed_size, 
                 encoder_hidden_size, decoder_hidden_size, reparameterize_size,
                 number_layers=2, bidirectional=True, dropout_rate=0.1,
                 glove_init=None):
        """Initializer."""
        super(VAE, self).__init__()
        self.indexer = indexer
        self.pad_index = self.indexer.get_index("PAD", add=False)
        self.start_symbol_index = self.indexer.get_index("<s>", add=False)
        self.end_symbol_index = self.indexer.get_index("</s>", add=False)
        vocab_size = self.indexer.size
        # Embedding lookup table.
        self.embed = Embeddings(embed_size, vocab_size, glove_init)
        # GRU encoder layer.
        self.encoder = GRUEncoder(embed_size, encoder_hidden_size)
        encoder_output_size = encoder_hidden_size * 2 \
            if bidirectional else encoder_hidden_size
        # Reparameterization linear layers.
        self.linear_mu = nn.Linear(encoder_output_size, reparameterize_size)
        self.linear_logvar = nn.Linear(encoder_output_size, reparameterize_size)
        self.linear_z = nn.Linear(reparameterize_size, decoder_hidden_size)
        # GRU decoder layer.
        self.decoder = GRUDecoder(embed_size, decoder_hidden_size)
        # Final linear layer (maps to vocabulary-size predictions).
        self.linear_final = nn.Linear(decoder_hidden_size, vocab_size)
    
    def forward(self, batch_source, batch_source_length, batch_target,
                enforce_rate=0.8):
        """Forwarding.
        
        Args:
            batch_source: torch.LongTensor, <max-encoder-length, batch-size>.
            batch_source_length: torch.LongTensor, <batch-size,>.
            batch_target: torch.LongTensor, <max-decoder-length, batch-size>.
            enforce_rate: proportion of data points where teacher enforce is imposed.
        Returns:
            prediction: torch.FloatTensor, <max-target-length, batch-size, vocab-size>.
        """
        batch_size = batch_source.size(-1)
        # Embed.
        #   IN: <max-encoder-length, batch-size>.
        #   OUT: <max-encoder-length, batch-size, embed-size>.
        batch_source_embedded = self.embed(batch_source) 
        batch_target_embedded = self.embed(batch_target)
        # Encode.
        #   IN: <max-encoder-length, batch-size, embed-size>, <batch-size,>.
        #   OUT: <number-layers, batch-size, encoder-hidden-size> (hidden*2 if bidirectional).
        batch_source_h = self.encoder(batch_source_embedded, batch_source_length)
        # Reparameterize.
        #   IN: <number-layers, batch-size, encoder-hidden-size>.
        #   OUT: <number-layers, batch-size, decoder-hidden-size>.
        mu, logvar = self.linear_mu(batch_source_h), self.linear_logvar(batch_source_h)
        sigma = torch.exp(logvar / 2)
        epsilon = torch.randn_like(sigma)
        z = mu + epsilon * sigma
        batch_source_z = self.linear_z(z)
        # Decode.
        decoding_length = batch_target.size(0)
        batch_decoder_input = self.embed(Var(torch.LongTensor([self.start_symbol_index] * batch_size))\
                                         .unsqueeze(0).to(DEVICE))
        batch_decoder_h = batch_source_z
        prediction = Var(torch.zeros(decoding_length, 
                                     batch_size, 
                                     self.indexer.size)).to(DEVICE)
        mask = torch.LongTensor([1] * batch_size).to(DEVICE)
        enforce = random.random() < enforce_rate
        for step in range(decoding_length):
            batch_decoder_output, batch_decoder_h = self.decoder(batch_decoder_input, batch_decoder_h)
            batch_decoder_output = self.linear_final(batch_decoder_output)
            prediction[step] = batch_decoder_output
            if enforce:
                batch_decoder_input = batch_target_embedded[step].unsqueeze(0)
            else:
                batch_top_value, batch_top_index = batch_decoder_output.topk(1) # both <batch-size,1>.
                batch_top_index = batch_top_index.squeeze()
                for element_index in range(batch_size):
                    if batch_top_index[element_index].item() == self.end_symbol_index:
                        mask[element_index] = 0
                batch_top_index *= mask
                batch_decoder_input = self.embed(batch_top_index).unsqueeze(0)
        # Prediction size: <max-target-length, batch-size, vocab-size>.
        return prediction, mu, logvar

##### TESTING AREA

In [363]:
vae = VAE(indexer, embed_size=300, encoder_hidden_size=100, decoder_hidden_size=100, reparameterize_size=100,
          number_layers=2, bidirectional=True, dropout_rate=0.1, glove_init=None)

crit = nn.CrossEntropyLoss()
out, mu, logvar = vae(s, sl, t)
pred = out.reshape(-1, vae.indexer.size)
true = t.reshape(-1)
r_loss = crit(pred, true)
k_loss = .5 * torch.sum(1. + logvar - mu.pow(2) - logvar.exp())
loss = r_loss + k_loss
loss.backward()

In [327]:
vocab_size = indexer.size
embed_size = 300
hidden_size = 100
batch_size = 3

emb = nn.Embedding(vocab_size, embed_size)
enc = GRUEncoder(embed_size, hidden_size)
lin_mu = nn.Linear(hidden_size * 2, hidden_size * 2)
lin_logvar = nn.Linear(hidden_size * 2, hidden_size * 2)
dec = GRUDecoder(embed_size, hidden_size * 2)
final = nn.Linear(hidden_size * 2, vocab_size)

# encode
s_emb = emb(s) # <mt-enc,bc,emb>
s_h = enc(s_emb, sl) # <n_layer,bc,hid*2>
# reparameterize
s_mu, s_logvar = lin_mu(s_h), lin_logvar(s_h) # <n_layer,bc,hid*2>
std = torch.exp(s_logvar / 2)
eps = torch.randn_like(std)
z = s_mu + eps * std # <n_layer,bc,hid*2>
# decode
dec_len = t.size(0)
t_emb = emb(t) # <mt-dec,bc,emb>
d_in = emb(Var(torch.LongTensor([0] * batch_size))).unsqueeze(0) # <mt=1,bc,emb>
d_h = z
d_all = Var(torch.zeros(dec_len, batch_size, vocab_size))
mask = torch.LongTensor([1] * batch_size)
enforce = random.random() > 0.5
for di in range(dec_len):
    d_o, d_h = dec(d_in, d_h)
    d_o = final(d_o)
    d_all[di] = d_o
    if enforce:
        d_in = t_emb[di].unsqueeze(0)
    else:
        tv, ti = d_o.topk(1)
        ti = ti.squeeze()
        for b in range(batch_size):
            if ti[b].item() == 3: # 3: </s>
                mask[b] = 0
        ti = ti * mask
        d_in = emb(ti).unsqueeze(0)
# compute loss
crit = nn.CrossEntropyLoss()
t_pred = d_all.reshape(-1, vocab_size)
t_true = t.reshape(-1)
loss = crit(t_pred, t_true)
loss.backward()

In [364]:
out.shape

torch.Size([14, 3, 22947])

In [370]:
out.argmax(dim=-1).transpose(0, 1).numpy()

array([[19591,  8898,  5984,  8759, 21087, 16261, 16261,  5382,  1067,
        18305, 15143, 13415, 12040, 20497],
       [20419, 15116,  3152,  5137,  9282, 11127, 11127, 14302, 11127,
        11127,  1665,  1920,  5902,  5382],
       [ 8362, 18761, 14519,  3152,  6587, 15352, 15876, 15876, 17627,
        16630, 19871,  9415, 10278, 11904]])

In [371]:
for sent in out.argmax(dim=-1).transpose(0, 1).numpy():
    print(vae.indexer.to_sent(sent))


unsliced daring comic films auxiliary presumed presumed escaltor arranged fishsticks drizzling emo lovers occasionally
julia claim signaling squatting neighbor diesel diesel mckee diesel diesel taste wears processes escaltor
important sakter froyo signaling dusting gunman drumming drumming lettuces bakset chalkboards fab hugged offs


In [376]:
for sent in t.transpose(0, 1).numpy():
    print(vae.indexer.to_words(sent))

['<s>', 'a', 'bear', 'walking', 'around', 'in', 'the', 'woods', 'looking', 'around', '</s>', 'PAD', 'PAD', 'PAD']
['<s>', 'a', 'pitcher', 'throwing', 'a', 'pitch', 'in', 'a', 'baseball', 'game', '</s>', 'PAD', 'PAD', 'PAD']
['<s>', 'a', 'smiling', 'woman', 'with', 'two', 'small', 'children', 'in', 'front', 'of', 'a', 'home', '</s>']


### Trainer

In [48]:
def remove_special_tokens(words):
    return " ".join([word for word in words
                     if word not in ["PAD", "<s>", "</s>"]])


def tensor_to_words(indexer, tensor):
    """Convert prediction/true tensor to words."""
    if DEVICE.type == "cuda":
        tensor = tensor.cpu()
    batch_words = tensor.transpose(0, 1).numpy()
    batch_sentence = [remove_special_tokens(indexer.to_words(words)) for words in batch_words]
    return batch_sentence


def run_batch(model, criterion, optimizer,
              batch_source, batch_target, batch_source_length, batch_target_length,
              data_group, print_results=False):
    assert data_group in ["Train", "Valid"]
    outputs, mu, logvar = model(batch_source, batch_source_length, batch_target)
    prediction = outputs.reshape(-1, model.indexer.size)
    true = batch_target.reshape(-1)
    reconstruction_loss = criterion(prediction, true)
    kl_divergence = - 0.5 * torch.sum(1. + logvar - mu.pow(2) - logvar.exp())
    loss = reconstruction_loss + kl_divergence
    if data_group == "Train":
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if print_results:
        sentences_predicted = tensor_to_words(model.indexer, outputs.argmax(dim=-1))
        sentences_true = tensor_to_words(model.indexer, batch_target)
        print("\nPrinting a batch of %d sample results:\n" % len(sentences_predicted))
        for sentence_predicted, sentence_true in zip(sentences_predicted, sentences_true):
            print("PRED >> %s" % sentence_predicted)
            print("TRUE >> %s\n" % sentence_true)
    return loss.item()


def train(data_dir, info_dir,
          embed_size, encoder_hidden_size, decoder_hidden_size, reparameterize_size,
          number_layers, bidirectional, dropout_rate, 
          number_epochs, batch_size, learning_rate,
          save_path, load_path,
          valid_size=100, valid_every=100,
          print_every=10,
          glove_path=None):
    print("Loading data ... \n")
    indexer, train_source, train_target, valid_source, valid_target = load_corpus(data_dir)
    configs = {"embed_size": embed_size,
               "encoder_hidden_size": encoder_hidden_size,
               "decoder_hidden_size": decoder_hidden_size,
               "reparameterize_size": reparameterize_size,
               "number_layers": number_layers,
               "bidirectional": bidirectional,
               "dropout_rate": dropout_rate,
               "learning_rate": learning_rate}
    dill.dump(indexer, open(info_dir + "indexer.p", "wb"))
    dill.dump(configs, open(info_dir + "configs.p", "wb"))
    train_iterator = LengthAdaptiveIterator(train_source, train_target)
    if glove_path is None:
        glove_init = None
    else:
        glove_init = load_glove(glove_path, indexer, embed_size)
    model = VAE(indexer, embed_size, encoder_hidden_size, decoder_hidden_size,
                reparameterize_size, number_layers, bidirectional, dropout_rate, glove_init).to(DEVICE)
    if load_path is not None:
        model.load_state_dict(torch.load(load_path))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    global_step = 0
    current_epoch = 0
    best_valid_loss = np.inf
    start = time.time()
    while train_iterator.epoch < number_epochs:
        print("\nEpoch %d" % (current_epoch + 1))
        model.train()
        train_losses = []
        while train_iterator.epoch <= current_epoch:
            global_step += 1
            print_results = True if global_step % print_every == 0 else False
            batch_source, batch_target, \
            batch_source_length, batch_target_length = train_iterator.next_batch(batch_size)
            batch_source, batch_target = batch_source.to(DEVICE), batch_target.to(DEVICE)
            batch_source_length = batch_source_length.to(DEVICE)
            batch_target_length = batch_target_length.to(DEVICE)
            train_loss = run_batch(model, criterion, optimizer,
                                   batch_source, batch_target,
                                   batch_source_length, batch_target_length,
                                   data_group="Train", print_results=print_results)
            train_losses.append(train_loss)
            if global_step % print_every == 0:
                print("At step %d: loss = %.5f." % (global_step, np.mean(train_losses[-print_every:])))
            if global_step % valid_every == 0:
                print("\nRunning valid ...\n")
                model.eval()
                valid_iterator = LengthAdaptiveIterator(valid_source, valid_target)
                valid_losses = []
                for valid_step in range(valid_size):
                    print_results = False # True if valid_step % print_every == 0 else False
                    batch_source, batch_target, \
                    batch_source_length, batch_target_length = valid_iterator.next_batch(batch_size)
                    batch_source, batch_target = batch_source.to(DEVICE), batch_target.to(DEVICE)
                    batch_source_length = batch_source_length.to(DEVICE)
                    batch_target_length = batch_target_length.to(DEVICE)
                    valid_loss = run_batch(model, criterion, optimizer,
                                           batch_source, batch_target,
                                           batch_source_length, batch_target_length,
                                           data_group="Valid", print_results=print_results)
                    valid_losses.append(valid_loss)
                average_train_loss = np.mean(train_losses)
                average_valid_loss = np.mean(valid_losses)
                print("\nValidation (size = %d) results:" % valid_size)
                print("Train (average) loss = %.6f" % average_train_loss)
                print("Valid (average) loss = %.6f\n" % average_valid_loss)
                train_losses = []
                if average_valid_loss < best_valid_loss:
                    print("Saving model weights for best valid loss %.6f" % average_valid_loss)
                    best_valid_loss = average_valid_loss
                    torch.save(model.state_dict(), save_path)
                    print("Saved as %s\n" % save_path)
                model.train()
        current_epoch += 1

In [412]:
DEVICE

device(type='cuda')

In [ ]:
#TODO: gpu enabled
#TODO: compute bleu
#TODO: compute pinc
#TODO: wrapper # DONE

In [49]:
%%time

data_dir = "Data/"
info_dir = "Info/"
embed_size = 300
encoder_hidden_size = 100
decoder_hidden_size = 100
reparameterize_size = 100
number_layers = 2
bidirectional = True
dropout_rate = 0.1
number_epochs = 5
batch_size = 10
learning_rate = 1e-4
save_path = "SavedModels/vae-test.ckpt"
load_path = None # "SavedModels/vae-test.ckpt"
valid_size = 1000
valid_every = 1000
print_every = 100
glove_path = "Glove/glove.6B.300d.txt"

train(data_dir, info_dir,
    embed_size, encoder_hidden_size, decoder_hidden_size, reparameterize_size,
    number_layers, bidirectional, dropout_rate, 
    number_epochs, batch_size, learning_rate,
    save_path, load_path,
    valid_size, valid_every,
    print_every,
    glove_path)

Loading data ... 

... loading `Data/train_source.txt`
... loading `Data/train_target.txt`
... loading `Data/valid_source.txt`
... loading `Data/valid_target.txt`

Loading glove embeddings (size = 300)

... processed 100000 lines.
... processed 200000 lines.
... processed 300000 lines.

Done!

Loaded 18157 | OOV size = 4790


Epoch 1

Printing a batch of 10 sample results:

PRED >> buddha rinsing of
TRUE >> ted a guy and girl sitting on a couch drinking

PRED >> rippling simply
TRUE >> a man is on the ground with a tennis racquet

PRED >> fashinoned fashinoned
TRUE >> an orange and white cat sitting inside of a car

PRED >> hatch "pops"
TRUE >> two women sitting on a bench at a park

PRED >> worn giron
TRUE >> a man throwing a baseball away from him

PRED >> rower redish
TRUE >> a young girl in colorful snow clothing skis on the bunny slope

PRED >> mechanism
TRUE >> a right hand holding a donut covered in sprinkles

PRED >> astove trafic
TRUE >> a young blond woman swinging a tennis r


Validation (size = 1000) results:
Train (average) loss = 13.391853
Valid (average) loss = 5.043160

Saving model weights for best valid loss 5.043160
Saved as SavedModels/vae-test.ckpt


Printing a batch of 10 sample results:

PRED >> a a a a a a a a a a
TRUE >> open sandwich mayo on a plate with chips and soda

PRED >> a a a a a a
TRUE >> a tray with 3 coffee drinks and about 8 different doughnuts

PRED >> a a a a a a a a
TRUE >> a man walking behind a large elephant on a dirty field

PRED >> a a a a a a a a a a a
TRUE >> a man opens the oven door to check the food inside

PRED >> a a a a a a a
TRUE >> some giraffes hanging out together in a fenced area

PRED >> a a a a a a a
TRUE >> a laptop and some more items on a wooden desk

PRED >> a a a a a a a
TRUE >> the baseball player swings his bat but strikes out

PRED >> a a a a a a a a a
TRUE >> a room is full of young men and luggage

PRED >> a a a a a a a a a a a a
TRUE >> woman sitting in a oddly built bench while two other people s


Printing a batch of 10 sample results:

PRED >> a a a a a
TRUE >> a large kitchen with black appliances and a wooden floor

PRED >> a a a a a
TRUE >> a bunch of people on a wet street in the rain

PRED >> a a a a a a a a
TRUE >> black and white image of woman in a black dress sitting on a kitchen counter with head cropped

PRED >> a a a a a a
TRUE >> a woman takes a shot at a tennis ball

PRED >> a a a a a
TRUE >> a pink fire hydrant on a paved sidewalk next to a street

PRED >> a a a a a a
TRUE >> a man holding a partially peeled bananas standing like he's holding a bow

PRED >> a a a a a a
TRUE >> a display in a store filled with oranges and apples

PRED >> a a a a a a a a a
TRUE >> a black bird eating a piece of fruit on top of the snow

PRED >> a a a a a a
TRUE >> a baby or teddy bear themes cake on a table

PRED >> a a a a a
TRUE >> crowd watches as male tennis player reaches back to serve the ball

At step 2000: loss = 4.05908.

Running valid ...


Validation (size = 1000) resul


Printing a batch of 10 sample results:

PRED >> a a of of of a
TRUE >> a vintage tan muscle car holding two surfboards

PRED >> a a on a in a
TRUE >> two apples and a banana are on a table

PRED >> a a a a on a a
TRUE >> a tennis player leans back to serve the ball

PRED >> a a of a a a
TRUE >> a couple of boats are tied to posts in the water

PRED >> a a of a of in a
TRUE >> a canister of flowers are adorned with a heart

PRED >> a a of a a a a a
TRUE >> young man wearing a red shirt holding a tennis racquet while standing on a tennis court

PRED >> a a on a a a a a
TRUE >> the man is playing the sport tennis with a racket

PRED >> a a of in a
TRUE >> two sheep are laying on the barn floor

PRED >> a a of a a a a a
TRUE >> small boy with white shirt on riding a skateboard

PRED >> a a of a a a a
TRUE >> a person who is wearing snow skis on a snowy hill

At step 2900: loss = 3.73535.

Printing a batch of 10 sample results:

PRED >> a man of a a a a
TRUE >> a man holding a tennis racqu


Printing a batch of 10 sample results:

PRED >> a man of on a a a a a
TRUE >> a dog leaning on a fence trying to get to a horse

PRED >> a man of a on on a a
TRUE >> a flock of seagulls standing on cement in a parking lot

PRED >> a man of on a a on a white a a
TRUE >> a kitchen area of a bakery with a baker looking into a double door glass oven at food that is baking

PRED >> a man of a of on a a
TRUE >> a bunch of birds stand an fly over a beach

PRED >> a man of of a a a
TRUE >> a cat looks at some items hanging on a string

PRED >> a of on on a a a
TRUE >> kids ride skateboards and try tricks at the skate park

PRED >> a man of of on on a a
TRUE >> a very dimly lit area with a very pretty sink

PRED >> a man of of on of a a
TRUE >> the british railways crest painted on the door of an old automobile

PRED >> a on on on a a
TRUE >> sheep grazing together on a grassy hill together

PRED >> a man of a a a a
TRUE >> a boy is playing a skateboard in a park

At step 3800: loss = 3.52655.

...


Printing a batch of 10 sample results:

PRED >> a man is standing in a fence tree
TRUE >> the giraffe is standing beside a large bush

PRED >> a man plate is at a stop station a
TRUE >> a white bus waits at a pedestrian crossing area

PRED >> a is a people of a to a tree
TRUE >> there are some dishes draining next to a sink

PRED >> a man is sitting on a bed in a room car
TRUE >> a dog is sitting on a couch in a motor home

PRED >> a man and white photo sitting a bed in
TRUE >> a black and white cat on a coach

PRED >> a man is a track in a street body
TRUE >> a train crosses a bridge over a peaceful river

PRED >> a man standing standing in to a tree
TRUE >> a giraffe is standing next to a warthog

PRED >> a man man is boarding on a ramp
TRUE >> a young boy skate boarding on a road

PRED >> a man of baseball standing a skiing down
TRUE >> a team of people cross country skiing together

PRED >> a man of people flying through up a in trees
TRUE >> a bunch of planes flying close with tr


Printing a batch of 10 sample results:

PRED >> a man of a a a a a a
TRUE >> a hand holding an orange carrot and a potato

PRED >> a man is a a a a a
TRUE >> a toilet with jelly fish and star fish on it

PRED >> a man of a a a a a a
TRUE >> a large bus sitting on top of a parking lot

PRED >> a man of a a a a a
TRUE >> a person holding a surf board walking into a body of water

PRED >> a man is a a a a a
TRUE >> a baseball player in uniform holding a baseball bat

PRED >> a man of a a a a a
TRUE >> a man has his mouth open with a large pizza slice

PRED >> a man is a a a a a a
TRUE >> a dog carries a tennis ball in it's mouth

PRED >> a man is a a a a a a
TRUE >> a man kneeling down to a stove while cooking food

PRED >> a man of a a a a a
TRUE >> a laptop and laptop bag surrounded by pencils calculator and other accessories

PRED >> a man of a a a a a
TRUE >> a cat that is standing on a laptop

At step 143200: loss = 2.56926.

Printing a batch of 10 sample results:

PRED >> a man is 


Printing a batch of 10 sample results:

PRED >> a man is a to a camera of a building bus
TRUE >> a woman gives bananas at the entrance of a tour bus

PRED >> a man in a umbrella in down a street
TRUE >> a man holding an umbrella walks along the path

PRED >> a man is in a large holding a a cake cake
TRUE >> a woman stands beside a man cutting into a chocolate cake

PRED >> a pizza with on a and other on a cream
TRUE >> baked dessert item with fruit and nuts and whipped topping displayed

PRED >> a view of a man that a hat tie on a bench floor
TRUE >> blurred image of a dog wearing a neck tie sitting on a wood table top

PRED >> a man of people types on on a middle
TRUE >> a group of different mopeds sitting in the street

PRED >> a man sign with a and a street a clock
TRUE >> a stop light on red by some bushes and a tree

PRED >> a man with with a bowl with food food a
TRUE >> a spoon blacked in a bowl of soup containing noodles peppers and green onions

PRED >> a man man is on to a m


Printing a batch of 10 sample results:

PRED >> a man with with a and window can a
TRUE >> a floor toilet some legs a trash can and white tiles

PRED >> a man giraffe riding next a tennis with a dogs a water boards a ocean
TRUE >> a lone man stands on a beach holding two surfboards near the surf of the beach

PRED >> a man on the be sign the the of a lines
TRUE >> a sign stating to stop all ways in front of power lines

PRED >> a man bear sitting in a water the zoo
TRUE >> a polar bear swimming under the water at the zoo

PRED >> a man of people are down a fence of water
TRUE >> a number of elephants walking towards a body of water

PRED >> a man on is sitting a on a tennis of pizza
TRUE >> a person who is holding scissors to a piece of paper

PRED >> a man player swinging a bat in a field
TRUE >> a baseball player holding his bat on the field

PRED >> a man with a of her animals bears
TRUE >> a lady with hundreds of stuffed teddy bears

PRED >> a man is sitting with a head on the bac

KeyboardInterrupt: 